In [19]:
import json
import os
import re
import subprocess
import types

from collections import abc
from functools import cached_property, partial
from itertools import chain
from pprint import pprint
from time import sleep

# pip install 'mido[ports-rtmidi]'
import mido
print(mido.backend)
# If this backend doesn't work for you, try another one:
# https://mido.readthedocs.io/en/stable/backends/index.html#choice

# pip install tabulate
from tabulate import tabulate

<backend mido.backends.rtmidi (loaded)>


In [2]:
from pypewyre import PWDump, PWState, PWObject, volume_to_linear, volume_from_linear

## Exploration and learning

In [2]:
def pw_dump():
    p = subprocess.run(['pw-dump', '--no-colors'], capture_output=True)
    return json.loads(p.stdout)

In [3]:
RE_AUDIO_SOURCE_SINK = re.compile(r"^Audio/(Source|Sink)")

def pw_filter(dump:list, id:set=None, type:str=None, media_class:re.Pattern=None):
    if type is not None and ":" not in type:
        # For convenience, let's this prefix.
        type = "PipeWire:Interface:" + type
        # PipeWire:Interface:Client
        # PipeWire:Interface:Core
        # PipeWire:Interface:Device
        # PipeWire:Interface:Factory
        # PipeWire:Interface:Link
        # PipeWire:Interface:Metadata
        # PipeWire:Interface:Module
        # PipeWire:Interface:Node
        # PipeWire:Interface:Port
        # PipeWire:Interface:Profiler

    for obj in dump:
        if id is not None:
            if not (obj["id"] in id):
                continue
        if type is not None:
            if not (obj["type"] == type):
                continue
        if media_class is not None:
            if not media_class.match(obj["info"]["props"].get("media.class", "")):
                continue
        yield obj

In [362]:
tmp = pw_dump()
len(tmp)

134

In [363]:
print("\n".join(sorted(set(x["type"] for x in tmp))))

PipeWire:Interface:Client
PipeWire:Interface:Core
PipeWire:Interface:Device
PipeWire:Interface:Factory
PipeWire:Interface:Link
PipeWire:Interface:Metadata
PipeWire:Interface:Module
PipeWire:Interface:Node
PipeWire:Interface:Port
PipeWire:Interface:Profiler


In [364]:
print("\n".join(sorted(set(x.get("info", {}).get("props", {}).get("media.class", "?") for x in tmp))))

?
Audio/Device
Audio/Sink
Audio/Source
Audio/Source/Virtual
Midi/Bridge
Stream/Input/Audio
Stream/Output/Audio
Video/Device
Video/Source


In [365]:
print("\n".join(sorted(set(x["info"]["props"].get("media.class", "?") for x in pw_filter(tmp, type="Node")))))

?
Audio/Sink
Audio/Source
Audio/Source/Virtual
Midi/Bridge
Stream/Input/Audio
Stream/Output/Audio
Video/Source


In [366]:
tmp = list(pw_filter(tmp, type="Node"))
len(tmp)

19

In [9]:
tmp = list(pw_filter(tmp, media_class=RE_AUDIO_SOURCE_SINK))
len(tmp)

10

In [10]:
tabulate(sorted([
    (
        x["info"]["props"].get("media.class", "N/A"),
        x["id"],
        x["info"]["props"].get("object.id", "N/A"),
        #x["info"]["props"].get("node.name", "N/A"),
        x["info"]["props"].get("node.description", "N/A"),
        x["info"]["props"].get("node.nick", "N/A"),
        x["info"]["props"].get("audio.channels", "N/A"),  # Sometimes unset.
        x["info"]["props"].get("audio.position", "N/A"),
        #x["info"]["params"].get("Props", [])[0].get("volume", "N/A"),  # Useless, always 1.
        x["info"]["params"].get("Props", [])[0].get("channelVolumes", "N/A"),
        x["info"]["params"].get("Props", [])[0].get("softVolumes", "N/A"),
        x["info"]["params"].get("Props", [])[0].get("mute", "N/A"),
        x["info"]["params"].get("Props", [])[0].get("softMute", "N/A"),
        #x["info"]["params"].get("Props", [])[0].get("monitorMute", "N/A"),
        #x["info"]["params"].get("Props", [])[0].get("monitorVolumes", "N/A"),
    )
    for x in tmp
]), tablefmt="html")

Audio/Sink,30,30,Null Stereo Output,N/A,N/A,[ FL FR ],"[1.0, 1.0]","[1.0, 1.0]",False,False
Audio/Sink,60,60,Built-in Audio Analog Stereo,ALC221 Analog,2,"FL,FR","[0.0, 0.0]","[0.0, 0.0]",False,False
Audio/Sink,61,61,Built-in Audio Digital Stereo (HDMI),Q27P1B,2,"FL,FR","[0.008, 0.008]","[0.008, 0.008]",False,False
Audio/Sink,62,62,Built-in Audio Digital Stereo (HDMI 2),Q27P1B,2,"FL,FR","[0.0, 0.0]","[0.0, 0.0]",True,True
Audio/Sink,63,63,Built-in Audio Digital Stereo (HDMI 3),HDMI 2,2,"FL,FR","[1.0, 1.0]","[1.0, 1.0]",False,False
Audio/Sink,97,97,Aeropex by AfterShokz,N/A,N/A,N/A,"[0.019188, 0.019188]","[1.0, 1.0]",False,False
Audio/Source,58,58,Creative Live! Cam Sync 1080p V2 Analog Stereo,Creative Live! Cam Sync 1080p V,2,"FL,FR","[0.0, 0.0]","[0.0, 0.0]",False,False
Audio/Source,59,59,Logitech USB Microphone Mono,Logitech USB Microphone,1,MONO,[1.0],[1.0],False,False
Audio/Source,64,64,Built-in Audio Analog Stereo,ALC221 Analog,2,"FL,FR","[1.0, 1.0]","[1.0, 1.0]",False,False
Audio/Source/Virtual,31,31,Null Mono Input,N/A,N/A,[ MONO ],[1.0],[1.0],False,False


In [13]:
volume_from_linear(0.5), volume_to_linear(volume_from_linear(0.5))

(0.125, 0.5)

In [12]:
def terrible_get_volume(ids:set):
    for obj in pw_filter(pw_dump(), id=ids):
        yield(
            obj["info"]["params"].get("Props", [])[0].get("volume", "N/A"),
            obj["info"]["params"].get("Props", [])[0].get("channelVolumes", "N/A"),
            obj["info"]["params"].get("Props", [])[0].get("softVolumes", "N/A"),
            volume_to_linear(obj["info"]["params"].get("Props", [])[0].get("channelVolumes", "N/A")),
            volume_to_linear(obj["info"]["params"].get("Props", [])[0].get("softVolumes", "N/A")),
            obj["info"]["params"].get("Props", [])[0].get("mute", "N/A"),
            obj["info"]["params"].get("Props", [])[0].get("softMute", "N/A"),
            obj["info"]["params"].get("Props", [])[0].get("monitorMute", "N/A"),
            obj["info"]["params"].get("Props", [])[0].get("monitorVolumes", "N/A"),
        )

In [14]:
tabulate(
    terrible_get_volume({30, 61, 62, 97})
, tablefmt="html")

1,"[1.0, 1.0]","[1.0, 1.0]","[1.0, 1.0]","[1.0, 1.0]",False,False,False,"[1.0, 1.0]"
1,"[0.008, 0.008]","[0.008, 0.008]","[0.2, 0.2]","[0.2, 0.2]",False,False,False,"[1.0, 1.0]"
1,"[0.0, 0.0]","[0.0, 0.0]","[0, 0]","[0, 0]",True,True,False,"[1.0, 1.0]"
1,"[0.019188, 0.019188]","[1.0, 1.0]","[0.2677173823275574, 0.2677173823275574]","[1.0, 1.0]",False,False,False,"[1.0, 1.0]"


In [16]:
# This works for setting the volume of applications (Stream/Input/Audio and Stream/Output/Audio).
def terrible_set_volume(id, channels=None, linear:float=None, raw:float=None):
    # TODO: Support "mute".
    # TODO: Support specific channels.
    if linear is None and raw is None:
        raise ValueError("Exactly one of `linear` and `raw` parameters should be given, and none were passed.")
    if linear is not None and raw is not None:
        raise ValueError("Exactly one of `linear` and `raw` parameters should be given, and both were passed.")

    obj = next(pw_filter(pw_dump(), id={id}))
    channelVolumes = obj["info"]["params"].get("Props", [])[0].get("channelVolumes")

    if raw is None:
        raw = volume_from_linear(linear)

    assert raw is not None
    raw = max(0.0, raw)

    props = {}

    if False:  # TODO: Mute
        props["mute"] = False

    props["channelVolumes"] = [raw for i in channelVolumes]
    #props["softVolumes"] = props["channelVolumes"]

    print(json.dumps(props))

    subprocess.run(
        ["pw-cli", "set-param", str(id), "Props", json.dumps(props)]
    )

In [159]:
terrible_set_volume(122, linear=1.0)

{"channelVolumes": [1.0, 1.0]}
Object: size 40, type Spa:Pod:Object:Param:Props (262146), id Spa:Enum:ParamId:Props (2)
  Prop: key Spa:Pod:Object:Param:Props:channelVolumes (65544), flags 00000000
    Array: child.size 4, child.type Spa:Float
      Float 1,000000
      Float 1,000000


In [17]:
# This works for setting the volume of devices/ports (Audio Sinks and Sources).
def terrible2_set_volume(id, *, channels:set=None, linear:float=None, raw:float=None, mute:bool=None):
    # Ideas:
    # * Get rid of "raw". No one cares. And it simplifies the code a lot.
    # * Pass the volume as either:
    #     * float: applies to all channels, e.g. 0.50
    #     * dict: applies to each channel, e.g. { "FR": 0.50, "FL": 0.25 }
    # * Pass two volume parameters: one for relative amounts, another for absolute amounts.
    # * For relative amounts, also pass an optional limit (that clamps to 100%).
    # * Range is still from 0.0 to 1.0 mapping from 0% to 100%.
    if mute is None:
        if linear is None and raw is None:
            raise ValueError("Exactly one of `linear` and `raw` parameters should be given, and none were passed.")
        if linear is not None and raw is not None:
            raise ValueError("Exactly one of `linear` and `raw` parameters should be given, and both were passed.")

    if raw is None and linear is not None:
        raw = volume_from_linear(linear)
    if raw is not None:
        raw = max(0.0, raw)

    dump = pw_dump()
    obj = next(pw_filter(dump, id={id}))
    device_id = obj["info"]["props"]["device.id"]
    dev = next(pw_filter(dump, id={device_id}))

    for route in dev["info"]["params"]["Route"]:
        if all(x in route for x in ["info", "props", "device", "index"]):
            route_index = route["index"]
            route_device = route["device"]
            route_props = route["props"]
            break
    else:
        raise RuntimeError("Could not find the route for node {} device {}".format(id, device_id))

    # pw-cli s <device-id> Route '{ index: <route-index>, device: <card-profile-device>, props: { mute: false, channelVolumes: [ 0.5, 0.5 ] }, save: true }'

    channelVolumes = route_props["channelVolumes"]
    channelMap = route_props["channelMap"]

    props = {}

    if mute is not None:
        props["mute"] = mute
    if raw is not None:
        newVolumes = [
            raw if channels is None or ch_name in channels else old_vol
            for (ch_name, old_vol) in zip(channelMap, channelVolumes)
        ]
        props["channelVolumes"] = newVolumes

    new_value = {
        "index": route_index,
        "device": route_device,
        "props": props,
        "save": True,
    }
    pprint(json.dumps(new_value))

    subprocess.run(
        ["pw-cli", "set-param", str(device_id), "Route", json.dumps(new_value)]
    )

In [141]:
for i in range(10, 51, 5):
    terrible2_set_volume(97, channels={"FL"}, linear=i/100, mute=bool(i%2))
    sleep(0.125)

('{"index": 1, "device": 1, "props": {"mute": false, "channelVolumes": '
 '[0.0010000000000000002, 0.091125]}, "save": true}')
Object: size 160, type Spa:Pod:Object:Param:Route (262153), id Spa:Enum:ParamId:Route (13)
  Prop: key Spa:Pod:Object:Param:Route:index (1), flags 00000000
    Int 1
  Prop: key Spa:Pod:Object:Param:Route:device (3), flags 00000000
    Int 1
  Prop: key Spa:Pod:Object:Param:Route:props (10), flags 00000000
    Object: size 64, type Spa:Pod:Object:Param:Props (262146), id Spa:Enum:ParamId:Route (13)
      Prop: key Spa:Pod:Object:Param:Props:mute (65540), flags 00000000
        Bool false
      Prop: key Spa:Pod:Object:Param:Props:channelVolumes (65544), flags 00000000
        Array: child.size 4, child.type Spa:Float
          Float 0,001000
          Float 0,091125
  Prop: key Spa:Pod:Object:Param:Route:save (13), flags 00000000
    Bool true
('{"index": 1, "device": 1, "props": {"mute": true, "channelVolumes": '
 '[0.0033749999999999995, 0.091125]}, "save": t

In [156]:
terrible2_set_volume(97, linear=1)

'{"index": 1, "device": 1, "props": {"channelVolumes": [1, 1]}, "save": true}'
Object: size 136, type Spa:Pod:Object:Param:Route (262153), id Spa:Enum:ParamId:Route (13)
  Prop: key Spa:Pod:Object:Param:Route:index (1), flags 00000000
    Int 1
  Prop: key Spa:Pod:Object:Param:Route:device (3), flags 00000000
    Int 1
  Prop: key Spa:Pod:Object:Param:Route:props (10), flags 00000000
    Object: size 40, type Spa:Pod:Object:Param:Props (262146), id Spa:Enum:ParamId:Route (13)
      Prop: key Spa:Pod:Object:Param:Props:channelVolumes (65544), flags 00000000
        Array: child.size 4, child.type Spa:Float
          Float 1,000000
          Float 1,000000
  Prop: key Spa:Pod:Object:Param:Route:save (13), flags 00000000
    Bool true


## Exploring live-updates from `pw-dump`

In [359]:
pwd = PWDump()

In [356]:
for j in pwd:
    print(type(j), len(j))

<class 'str'> 5


In [360]:
pwd

<PWDump('pw-dump') pid=None, fileno=None>

## Exploring MIDO

In [8]:
mido.get_input_names()

['Midi Through:Midi Through Port-0 14:0',
 'Arduino Leonardo:Arduino Leonardo MIDI 1 28:0']

In [9]:
mido.get_output_names()

['Midi Through:Midi Through Port-0 14:0',
 'Arduino Leonardo:Arduino Leonardo MIDI 1 28:0',
 'Client-128:qpwgraph_alsamidi 128:0']

In [10]:
mido.get_ioport_names()

['Midi Through:Midi Through Port-0 14:0',
 'Arduino Leonardo:Arduino Leonardo MIDI 1 28:0']

In [20]:
def mido_callback(port, message):
    print(port, repr(message))

In [21]:
pi = mido.open_input(name="Arduino Leonardo:Arduino Leonardo MIDI 1 28:0", callback=partial(mido_callback, "Arduino"))

Arduino Message('control_change', channel=0, control=0, value=92, time=0)
Arduino Message('control_change', channel=0, control=0, value=93, time=0)
Arduino Message('control_change', channel=0, control=0, value=92, time=0)
Arduino Message('control_change', channel=0, control=0, value=93, time=0)
Arduino Message('control_change', channel=0, control=0, value=93, time=0)
Arduino Message('control_change', channel=0, control=0, value=94, time=0)
Arduino Message('control_change', channel=0, control=0, value=94, time=0)
Arduino Message('control_change', channel=0, control=0, value=95, time=0)
Arduino Message('control_change', channel=0, control=0, value=96, time=0)
Arduino Message('control_change', channel=0, control=0, value=96, time=0)
Arduino Message('control_change', channel=0, control=0, value=97, time=0)
Arduino Message('control_change', channel=0, control=0, value=98, time=0)
Arduino Message('control_change', channel=0, control=0, value=98, time=0)
Arduino Message('control_change', chan

In [22]:
pi.close()

## Next steps

* [ ] Create a git repository.
* [ ] Glue together PWState and PWDump.
* [ ] Write a nice set_volume function that uses the correct logic depending on the object.
* [ ] Write a shitload of unit tests.
* [x] Explore MIDI and MIDO.

## Older next steps

* [ ] Write a terrible function that changes the absolute volume of a certain device (e.g. by id). It's a proof-of-concept that should work perfectly.
* [ ] Allow changing it to relative amounts. (Why?)
* [x] Allow changing it beyond the 100% bounds.
* [ ] Stop and think. And probably create a git repository. Things are starting to work.
* [ ] Explore MIDO.
* [ ] Listen for MIDI events (even without connecting to anything).
* [ ] Figure out how to auto-connect to a certain device.
* [ ] Figure out how to listen to creation of MIDI devices, so we can try auto-connecting on-the-fly.
* [ ] Celebrate.
* [ ] Refactor everything. Try to tidy it up. Try to make it easy to map one thing to another.

## Further steps

For each `Node`, find its `device.id`. Then, for the `Device` object, look for each `Route`. Each route goes to a node, or something like that.

Having all that information, it should be possible to change the volume:

```
pw-cli set-param <device-id> Route '{ index: <route-index>, device: <card-profile-device>, props: { mute: false, channelVolumes: [ 0.5, 0.5 ] }, save: true }'
```

Where stuff inside `props` can be:

* channelVolumes
* mute
* monitorVolumes
* monitorMute
* And other low-level stuff I don't want to touch (such as latency)

---

Alternatively, we can try this as well:

```
pw-cli set-param <node-id> Props '{ mute: false, channelVolumes: [ 0.3, 0.3 ] }'
```

Which should be much easier and straighforward.

### Later steps

* Figure out how to listen to MIDI events.
* Figure out a good configuation format. Maybe even a GUI (probably not).

## Links

* WirePlumbler-related:
    * https://wiki.archlinux.org/title/WirePlumber#Keyboard_volume_control
    * https://unix.stackexchange.com/questions/758970/how-to-check-adjust-sound-volume-from-the-command-line-with-pipewire-pulse , which just suggests `wpctl set-volume @DEFAULT_SINK@ .03-` and `wpctlstatus`
    * https://github.com/PipeWire/wireplumber/blob/master/src/tools/wpctl.c
    * https://github.com/PipeWire/wireplumber/blob/master/modules/module-mixer-api.c
    * https://www.reddit.com/r/swaywm/comments/w3llwx/getting_volume_level_from_pipewire/ , which just suggests `wpctl get-volume @DEFAULT_AUDIO_SINK@` or `@DEFAULT_AUDIO_SOURCE@` for mics.
    * Getting a list of devices: https://github.com/PipeWire/wireplumber/blob/master/src/tools/shell-completion/wpctl.zsh#L8-L20
* Pure PipeWire:
    * https://gitlab.freedesktop.org/pipewire/pipewire/-/wikis/Migrate-PulseAudio#sinksource-port-volumemuteport-latency
        * OMG it's so complicated!
    * https://github.com/smasher164/pw-volume/blob/main/src/main.rs
        * `pw-cli set-param <id> Route {some complicated JSON}`
    * https://gitlab.freedesktop.org/pipewire/pipewire/-/issues/1654 non-working Python bindings https://gitlab.com/matpi/pipewire-py
* MIDI to control volume:
    * https://unix.stackexchange.com/questions/297449/regulate-system-volume-with-midi-controller (no answers yet)
    * https://superuser.com/questions/1170136/translating-midi-input-into-computer-keystrokes-on-linux , which uses `aseqdump` with `xdotool`
    * https://gitlab.com/enetheru/midi2input , which uses Lua and C++ to convert MIDI to arbitrary inputs.
    * https://github.com/Avante-Vangard/AV-MidiMacros , ugh, shell and `xdotool` and a bunch of CSV files.
    * https://github.com/omriharel/deej - Custom hardware to do it.